In [19]:
'''
　将ｍｈｄ图像转化为ｔｉｆ格式
'''
import cv2
import numpy as np
import os
import SimpleITK as itk
import pydicom
import shutil
import tensorflow as tf
from PIL import Image
from libtiff import TIFF

In [10]:
def image_erode(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.erode(image, kernel)
    return image


def image_expand(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.dilate(image, kernel)
    return image

'''
    此ｂｌｏｃｋ封装一些常见的操作
'''
# 读取单个DICOM文件
def read_dicom_file(file_name):
    header = pydicom.read_file(file_name)
    image = header.pixel_array
    image = header.RescaleSlope * image + header.RescaleIntercept
    return image
# 读取mhd文件
def read_mhd_image(file_path):
    header = itk.ReadImage(file_path)
    image = itk.GetArrayFromImage(header)
    return np.array(image)


# 保存mhd文件
def save_mhd_image(image, file_name):
    header = itk.GetImageFromArray(image)
    itk.WriteImage(header, file_name)

In [11]:
mhd_image = read_mhd_image('/home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/val/2542235-1747625_5/PV_ROI_Expand.mhd')

In [47]:
print np.max(mhd_image)
print np.min(mhd_image)
print type(mhd_image[0, 0])

136.0
0.0
<type 'numpy.float64'>


In [49]:
def convert_png(roi_dir):
    '''
        把所有的mhd格式的ROI转变成ＰＮＧ格式
    '''
    names = os.listdir(roi_dir)
    paths = [os.path.join(roi_dir, name) for name in names]
    for path in paths:
        print 'doing ', path
        for phase_name in ['NC', 'ART', 'PV']:
            mhd_path = os.path.join(path, phase_name+'_ROI_Expand.mhd')
            mhd_image = read_mhd_image(mhd_path)
            mhd_image[mhd_image<0] = 0
            tiff_image = Image.fromarray(np.asarray(mhd_image, np.uint8))
            tiff_image.save(os.path.join(path, phase_name+'_ROI_Expand.png'))
convert_png('/home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train')

doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/2894403-2144517_4
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3498105-3010128_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3807435-3477702_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3060906-2372564_4
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3036769-2337412_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3353017-2859533_1
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3278168-2709037_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3615324-3219324_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3067502-2381826_4
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/2956711-2810792_1


doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3166432-2525487_4
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3565759-3120933_4
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3375468-2824331_2
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/2838798-2082585_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3289366-2789100_2
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3556545-3100959_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3574812-3138306_1
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3085944-2406122_5
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/3370534-2827401_3
doing  /home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train/8152526-3177106_5


In [45]:
mhd_image[mhd_image<0] = 0
tiff_image = Image.fromarray(np.asarray(mhd_image, np.uint8))
tiff_image.save('./tiff_image.png')

In [46]:
def test_save():
    tiff_img = Image.open('./tiff_image.png')
    tiff_image = np.array(tiff_img)
    print np.max(tiff_image)
    print np.min(tiff_image)
    print type(tiff_image[0,0])
test_save()

136
0
<type 'numpy.uint8'>


In [42]:
def tensorflow_decode_tiff():
    path = './tiff_image.png'
    # path = '/home/give/Documents/dataset/AIChallenger/ai_challenger_scene_train_20170904/test/0016a32b7feaed0191aa5fc8ca2e044e8f6d4503.jpg'
    image_buffer = tf.read_file(path)
    image = tf.image.decode_png(image_buffer, channels=3)
    print tf.shape(image)
    sess = tf.Session()
    image_arr = image.eval(session=sess)
    print np.max(image_arr)
    print np.min(image_arr)
tensorflow_decode_tiff()

Tensor("Shape_9:0", shape=(3,), dtype=int32)
0
0
